# 2b2t DCF Valuation

## Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# set global plot settings
plt.rcParams['figure.dpi'] = 250
plt.style.use('ggplot')

Matplotlib is building the font cache; this may take a moment.


## ——— INPUTS ———

In [2]:

# 1) Business cash flow inputs
paying_players            = 4000    # number of monthly subscribers
price_per_player_per_month = 20     # $/month

# Monthly operating costs
hosting_per_month         = 1500    # server + bandwidth
plugin_maint_per_month    = 3000    # bug-fix & minor updates
admin_labor_per_month     = 5000    # support/admin staff

# Annual non-cash & capex items
depreciation_per_year     = 2000   # hardware depreciation + amortization of capitalized dev
plugin_dev_capex_per_year = 25000  # new feature builds (capitalized)
hardware_capex_per_year   = 2000   # server replacements, upgrades

# Taxes & working capital
tax_rate                  = 0.21    # corporate rate (set to 0 if not taxed)
delta_working_capital     = 25000   # e.g. prepaid hosting, unbilled dev work

# 2) DCF forecast inputs
forecast_years             = 10
growth_rate_annual         = 0.05      # 5% growth in FCF each year
terminal_growth_rate       = 0.02      # 2% perpetual growth after year 5

# 3) WACC inputs (for discount rate)
risk_free_rate             = 0.04      # risk-free rate (e.g. 10-yr Treasury)
equity_beta                = 2       # high beta due to small and risky business
equity_market_premium      = 0.06      # market risk premium
cost_of_debt               = 0.05      # pre-tax cost of debt
debt_to_capital_ratio      = 0      # D / (D + E)
equity_to_capital_ratio    = 1 - debt_to_capital_ratio


## ——— STEP 1: COMPUTE BASE FCF ———

In [3]:

# ——— CALCULATIONS ———
# 1) Revenue
annual_revenue = paying_players * price_per_player_per_month * 12

# 2) Operating expenses (expensed)
annual_opex = (hosting_per_month + 
               plugin_maint_per_month + admin_labor_per_month) * 12

# 3) EBITDA & EBIT
ebitda = annual_revenue - annual_opex
ebit   = ebitda - depreciation_per_year

# 4) NOPAT
nopat = ebit * (1 - tax_rate)

# 5) Free Cash Flow
total_capex = plugin_dev_capex_per_year + hardware_capex_per_year
base_fcf = nopat + depreciation_per_year - total_capex - delta_working_capital

# ——— OUTPUT ———
df = pd.DataFrame({
    'Metric': [
        'Annual Revenue',
        'Annual Opex (expensed)',
        'EBITDA',
        'EBIT',
        'NOPAT',
        'Depreciation & Amortization',
        'Plugin Dev CapEx',
        'Hardware CapEx',
        'Δ Working Capital',
        'Free Cash Flow'
    ],
    'Amount ($)': [
        annual_revenue,
        annual_opex,
        ebitda,
        ebit,
        nopat,
        depreciation_per_year,
        plugin_dev_capex_per_year,
        hardware_capex_per_year,
        delta_working_capital,
        base_fcf
    ]
}).set_index('Metric')

# Format numbers as currency strings
df['Amount ($)'] = df['Amount ($)'].apply(lambda x: f"${x:,.2f}")

# Print the table
print(df.to_string())

                              Amount ($)
Metric                                  
Annual Revenue               $960,000.00
Annual Opex (expensed)       $114,000.00
EBITDA                       $846,000.00
EBIT                         $844,000.00
NOPAT                        $666,760.00
Depreciation & Amortization    $2,000.00
Plugin Dev CapEx              $25,000.00
Hardware CapEx                 $2,000.00
Δ Working Capital             $25,000.00
Free Cash Flow               $616,760.00


## ——— STEP 2: CALCULATE WACC ———

In [4]:
cost_of_equity = risk_free_rate + equity_beta * equity_market_premium
after_tax_cost_of_debt = cost_of_debt * (1 - tax_rate)

wacc = (equity_to_capital_ratio * cost_of_equity +
        debt_to_capital_ratio   * after_tax_cost_of_debt)

# output WACC
print(f"\nWACC: {wacc:.2%} (cost of equity: {cost_of_equity:.2%}, "
      f"cost of debt: {after_tax_cost_of_debt:.2%})")


WACC: 16.00% (cost of equity: 16.00%, cost of debt: 3.95%)


## ——— STEP 3: FORECAST FCF + TERMINAL VALUE ———

In [5]:

years = list(range(1, forecast_years + 1))
fcf_forecast = [base_fcf * (1 + growth_rate_annual) ** yr for yr in years]

# Terminal value at end of year N
terminal_value = fcf_forecast[-1] * (1 + terminal_growth_rate) / (wacc - terminal_growth_rate)

# Discount factors
discount_factors = [(1 + wacc) ** yr for yr in years]

# Present values of forecast FCFs
pv_fcf = [fcf / df for fcf, df in zip(fcf_forecast, discount_factors)]
pv_terminal = terminal_value / discount_factors[-1]

# aggregate present values
enterprise_value = sum(pv_fcf) + pv_terminal

## ——— OUTPUT: FCF BREAKDOWN ———

In [6]:

fcf_df = pd.DataFrame({
    'Year':         years + ['Terminal'],
    'FCF ($)':      fcf_forecast + [terminal_value],
    'Discount Factor': discount_factors + [discount_factors[-1]],
    'PV of Cash Flow ($)': pv_fcf + [pv_terminal]
}).set_index('Year')

# Format numbers for display
fcf_df['FCF ($)'] = fcf_df['FCF ($)'].apply(lambda x: f"${x:,.0f}")
fcf_df['Discount Factor'] = fcf_df['Discount Factor'].apply(lambda x: f"{x:.2f}")
fcf_df['PV of Cash Flow ($)'] = fcf_df['PV of Cash Flow ($)'].apply(lambda x: f"${x:,.0f}")

print("\n— 2b2t Server DCF Projection —")
print(fcf_df)
print(f"\nCalculated WACC: {wacc:.2%}")
print(f"Enterprise Value (PV of FCF + Terminal): ${enterprise_value:,.0f}")


— 2b2t Server DCF Projection —
             FCF ($) Discount Factor PV of Cash Flow ($)
Year                                                    
1           $647,598            1.16            $558,274
2           $679,978            1.35            $505,334
3           $713,977            1.56            $457,415
4           $749,676            1.81            $414,039
5           $787,159            2.10            $374,777
6           $826,517            2.44            $339,238
7           $867,843            2.83            $307,069
8           $911,235            3.28            $277,950
9           $956,797            3.80            $251,593
10        $1,004,637            4.41            $227,735
Terminal  $7,319,499            4.41          $1,659,210

Calculated WACC: 16.00%
Enterprise Value (PV of FCF + Terminal): $5,372,633


In [ ]:
# summary of 